In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold
import re
from sklearn.linear_model import LogisticRegression

/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.neighbors import KNeighborsClassifier as KN

In [17]:
from sklearn.metrics import accuracy_score


In [ ]:
#lodaing data
data_path = "/home/raku/kaggleData/2sigma/lr4/"
train_file = data_path + "lr4-n-train.json"
test_file = data_path + "lr4-n-test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)
pickl_file = data_path+'lr4features.pickle'
fileObject = open(pickl_file,'r') 
lrfeature = pickle.load(fileObject)   
fileObject.close()
print len(lrfeature)

In [7]:
statistical = []

for feature in lrfeature:
    if re.match('((manager_id)|(house_type))\S+((mean)|(median)|(min)|(max))',feature) !=None:
        statistical.append(feature)
        
weightFeatures=[i for i in lrfeature if i not in statistical ]

In [8]:
#features
with_feat = []
for feature in lrfeature:
    if re.match('with_\S+',feature) !=None:
        statistical.append(feature)

In [9]:
weightFeatures.remove('another_day')
weightFeatures.remove('another_hour')
weightFeatures.remove('another_pic_day')
weightFeatures.remove('another_pic_hour')


In [6]:
weightFeatures

[u'bath_per_bed',
 u'bathrooms',
 u'bedrooms',
 u'building0',
 u'cluster_id_10_d',
 u'cluster_id_30_d',
 u'dayofyear',
 u'latitude',
 u'listing_id',
 u'longitude',
 u'm_c_distance',
 u'm_m_distance',
 u'manager_id_nrank',
 u'manager_id_perf',
 u'mlat',
 u'mlon',
 u'num_description_words',
 u'num_features',
 u'num_photos',
 u'price',
 u'price_per_bath',
 u'price_per_bed',
 u'price_per_room',
 u'created_day',
 u'created_hour',
 u'created_month',
 u'time_stamp',
 'hcc_manager_id_high',
 'hcc_manager_id_medium',
 'hcc_house_type_high',
 'hcc_house_type_medium',
 'pic_month',
 'pic_day',
 'pic_hour',
 u'with_actual_apt._photos',
 u'with_balcony',
 u'with_bike_room',
 u'with_cats_allowed',
 u'with_central_a/c',
 u'with_childrens_playroom',
 u'with_common_outdoor_space',
 u'with_common_parking/garage',
 u'with_common_roof_deck',
 u'with_concierge',
 u'with_dining_room',
 u'with_dishwasher',
 u'with_dogs_allowed',
 u'with_doorman',
 u'with_dryer_in_unit',
 u'with_duplex',
 u'with_eat_in_kitche

In [71]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 2333)

#run the logistic algorithm to do 
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
cv_scores=[]
models =[]
for dev_index, val_index in KF:
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[weightFeatures].as_matrix(), val_set[weightFeatures].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    kn = ()
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)
    models.append(lr)   
    cv_scores.append(log_loss(val_y, preds))
        

    #print(cv_scores)
print np.mean(cv_scores)


KeyboardInterrupt: 

In [ ]:
coff = np.abs(models[0].coef_) 
for i in [1,2,3,4]:
    coff+=np.abs(models[i].coef_) 
coeff = coff.sum(axis=0)

total_weight={}
total_weight_list=[]
for i in range(len(weightFeatures)):
    total_weight[weightFeatures[i]]=coeff[i]
    total_weight_list.append((weightFeatures[i],coeff[i]))
#total_weight_list=sorted(total_weight_list,key=lambda x:x[1],reverse=True)

In [ ]:
#trying knn using the weight from the above regression
knn_train_df = train_df.copy()
knn_test_df = test_df.copy()

for feature_value in total_weight_list:
    knn_train_df[feature_value[0]]*=np.sqrt(feature_value[1])
    knn_test_df[feature_value[0]]*=np.sqrt(feature_value[1])

In [ ]:
store = "/home/raku/kaggleData/2sigma/knn4/"
knn_train_df.to_json('knn-train.json')
knn_test_df.to_json('knn-test.json')


pickl_file = store+'knn-weigthed-features.pickle'
fileObject = open(pickl_file,'wb') 
pickle.dump(weightFeatures,fileObject)   
fileObject.close()

In [59]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 2333)

In [60]:
#first edition:
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
#features=list(processing_features)+with_feat
cv_scores=[]
models =[]
for dev_index, val_index in KF:
    dev_set, val_set = knn_train_df.iloc[dev_index,:] , knn_train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    kn = KN()
    kn.fit(dev_X,dev_y)
    preds = kn.predict_proba(val_X)
        
    cv_scores.append(log_loss(val_y, preds))
        

    print(cv_scores)
    break
print np.mean(cv_scores)


[2.9333649797334811]
2.93336497973


In [64]:
accuracy_score(kn.predict(val_X),val_y)

0.71178198764056322

In [43]:
features

[u'bath_per_bed',
 u'bathrooms',
 u'bedrooms',
 u'building0',
 u'cluster_id_10_d',
 u'cluster_id_30_d',
 u'dayofyear',
 u'latitude',
 u'listing_id',
 u'longitude',
 u'm_c_distance',
 u'm_m_distance',
 u'manager_id_nrank',
 u'manager_id_perf',
 u'mlat',
 u'mlon',
 u'num_description_words',
 u'num_features',
 u'num_photos',
 u'price',
 u'price_per_bath',
 u'price_per_bed',
 u'price_per_room',
 u'created_day',
 u'created_hour',
 u'created_month',
 u'time_stamp',
 'hcc_manager_id_high',
 'hcc_manager_id_medium',
 'hcc_house_type_high',
 'hcc_house_type_medium',
 'pic_month',
 'pic_day',
 'pic_hour',
 u'with_actual_apt._photos',
 u'with_balcony',
 u'with_bike_room',
 u'with_cats_allowed',
 u'with_central_a/c',
 u'with_childrens_playroom',
 u'with_common_outdoor_space',
 u'with_common_parking/garage',
 u'with_common_roof_deck',
 u'with_concierge',
 u'with_dining_room',
 u'with_dishwasher',
 u'with_dogs_allowed',
 u'with_doorman',
 u'with_dryer_in_unit',
 u'with_duplex',
 u'with_eat_in_kitche

In [12]:
basic_features = ['bathrooms','bedrooms', 'latitude', 'longitude','listing_id','price','price_per_bed','price_per_room']

In [66]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 2333)

#run the logistic algorithm to do 
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
cv_scores=[]
models =[]
for dev_index, val_index in KF:
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[basic_features].as_matrix(), val_set[basic_features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    kn = ()
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)
    models.append(lr)   
    cv_scores.append(log_loss(val_y, preds))
        

    #print(cv_scores)
print np.mean(cv_scores)


0.72271469576


In [67]:
coff = np.abs(models[0].coef_) 
for i in [1,2,3,4]:
    coff+=np.abs(models[i].coef_) 
coeff = coff.sum(axis=0)

total_weight={}
total_weight_list=[]
for i in range(len(basic_features)):
    total_weight[basic_features[i]]=coeff[i]
    total_weight_list.append((basic_features[i],coeff[i]))
#total_weight_list=sorted(total_weight_list,key=lambda x:x[1],reverse=True)

In [68]:
#trying knn using the weight from the above regression
knn_train_df = train_df.copy()
knn_test_df = test_df.copy()

for feature_value in total_weight_list:
    knn_train_df[feature_value[0]]*=np.sqrt(feature_value[1])
    knn_test_df[feature_value[0]]*=np.sqrt(feature_value[1])

In [69]:
#first edition:
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
#features=list(processing_features)+with_feat
for dev_index, val_index in KF:
    dev_set, val_set = knn_train_df.iloc[dev_index,:] , knn_train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[basic_features].as_matrix(), val_set[basic_features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    kn = KN()
    kn.fit(dev_X,dev_y)
    preds = kn.predict_proba(val_X)
        
    cv_scores.append(log_loss(val_y, preds))
        

    print(cv_scores)
    break
print np.mean(cv_scores)

[0.71569973846561874, 0.7256302364557482, 0.70545173366351122, 0.73056992785278352, 0.73622184236144717, 3.5521508239977964]
1.1942873838


In [70]:
accuracy_score(kn.predict(val_X),val_y)

0.64745213250937084

In [14]:
basic_features.extend(['manager_id_perf','time_stamp'])
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 2333)

#run the logistic algorithm to do 
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
cv_scores=[]
models =[]
for dev_index, val_index in KF:
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[basic_features].as_matrix(), val_set[basic_features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    lr = LogisticRegression()
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)
    models.append(lr)   
    cv_scores.append(log_loss(val_y, preds))
        

    #print(cv_scores)
print np.mean(cv_scores)


/opt/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


0.63070601339


In [15]:
coff = np.abs(models[0].coef_) 
for i in [1,2,3,4]:
    coff+=np.abs(models[i].coef_) 
coeff = coff.sum(axis=0)

total_weight={}
total_weight_list=[]
for i in range(len(basic_features)):
    total_weight[basic_features[i]]=coeff[i]
    total_weight_list.append((basic_features[i],coeff[i]))
#total_weight_list=sorted(total_weight_list,key=lambda x:x[1],reverse=True)

#trying knn using the weight from the above regression
knn_train_df = train_df.copy()
knn_test_df = test_df.copy()

for feature_value in total_weight_list:
    knn_train_df[feature_value[0]]*=np.sqrt(feature_value[1])
    knn_test_df[feature_value[0]]*=np.sqrt(feature_value[1])

In [18]:
#first edition:
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
#features=list(processing_features)+with_feat
cv_scores=[]
models =[]

for dev_index, val_index in KF:
    dev_set, val_set = knn_train_df.iloc[dev_index,:] , knn_train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[basic_features].as_matrix(), val_set[basic_features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    kn = KN()
    kn.fit(dev_X,dev_y)
    #preds = kn.predict_proba(val_X)
    accr=accuracy_score(kn.predict(val_X),val_y)   
    #cv_scores.append(log_loss(val_y, preds))
    cv_scores.append(accr)  

    print accr
    break
print np.mean(cv_scores)

0.691621922804
0.691621922804


In [19]:
basic_features.extend(['hcc_manager_id_high','hcc_manager_id_medium'])
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 2333)

#run the logistic algorithm to do 
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
cv_scores=[]
models =[]
for dev_index, val_index in KF:
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[basic_features].as_matrix(), val_set[basic_features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    lr = LogisticRegression()
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)
    models.append(lr)   
    cv_scores.append(log_loss(val_y, preds))
        

    #print(cv_scores)
print np.mean(cv_scores)

0.619298970114


In [20]:
coff = np.abs(models[0].coef_) 
for i in [1,2,3,4]:
    coff+=np.abs(models[i].coef_) 
coeff = coff.sum(axis=0)

total_weight={}
total_weight_list=[]
for i in range(len(basic_features)):
    total_weight[basic_features[i]]=coeff[i]
    total_weight_list.append((basic_features[i],coeff[i]))
#total_weight_list=sorted(total_weight_list,key=lambda x:x[1],reverse=True)

#trying knn using the weight from the above regression
knn_train_df = train_df.copy()
knn_test_df = test_df.copy()

for feature_value in total_weight_list:
    knn_train_df[feature_value[0]]*=np.sqrt(feature_value[1])
    knn_test_df[feature_value[0]]*=np.sqrt(feature_value[1])

In [21]:
#first edition:
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
#features=list(processing_features)+with_feat
cv_scores=[]
models =[]

for dev_index, val_index in KF:
    dev_set, val_set = knn_train_df.iloc[dev_index,:] , knn_train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[basic_features].as_matrix(), val_set[basic_features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    kn = KN()
    kn.fit(dev_X,dev_y)
    #preds = kn.predict_proba(val_X)
    accr=accuracy_score(kn.predict(val_X),val_y)   
    #cv_scores.append(log_loss(val_y, preds))
    cv_scores.append(accr)  

    #print accr
    break
print np.mean(cv_scores)

0.690507547361


In [22]:
basic_features.remove('manager_id_perf')


In [29]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 2333)

#run the logistic algorithm to do 
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
cv_scores=[]
models =[]
for dev_index, val_index in KF:
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[basic_features].as_matrix(), val_set[basic_features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    lr = LogisticRegression()
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)
    models.append(lr)   
    cv_scores.append(log_loss(val_y, preds))
        

    #print(cv_scores)
print np.mean(cv_scores)

0.619298352856


In [30]:
coff = np.abs(models[0].coef_) 
for i in [1,2,3,4]:
    coff+=np.abs(models[i].coef_) 
coeff = coff.sum(axis=0)

total_weight={}
total_weight_list=[]
for i in range(len(basic_features)):
    total_weight[basic_features[i]]=coeff[i]
    total_weight_list.append((basic_features[i],coeff[i]))
#total_weight_list=sorted(total_weight_list,key=lambda x:x[1],reverse=True)

#trying knn using the weight from the above regression
knn_train_df = train_df.copy()
knn_test_df = test_df.copy()

for feature_value in total_weight_list:
    knn_train_df[feature_value[0]]*=np.sqrt(feature_value[1])
    knn_test_df[feature_value[0]]*=np.sqrt(feature_value[1])

In [31]:
#first edition:
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
#features=list(processing_features)+with_feat
cv_scores=[]
models =[]

for dev_index, val_index in KF:
    dev_set, val_set = knn_train_df.iloc[dev_index,:] , knn_train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[basic_features].as_matrix(), val_set[basic_features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    kn = KN()
    kn.fit(dev_X,dev_y)
    #preds = kn.predict_proba(val_X)
    accr=accuracy_score(kn.predict(val_X),val_y)   
    #cv_scores.append(log_loss(val_y, preds))
    cv_scores.append(accr)  

    #print accr
    break
print np.mean(cv_scores)

0.692128457097


In [32]:
basic_features.extend([u'created_day',
 u'created_hour',
 u'created_month'])

In [33]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 2333)

#run the logistic algorithm to do 
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
cv_scores=[]
models =[]
for dev_index, val_index in KF:
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[basic_features].as_matrix(), val_set[basic_features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    lr = LogisticRegression()
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)
    models.append(lr)   
    cv_scores.append(log_loss(val_y, preds))
        

    #print(cv_scores)
print np.mean(cv_scores)

0.616317075081


In [34]:
coff = np.abs(models[0].coef_) 
for i in [1,2,3,4]:
    coff+=np.abs(models[i].coef_) 
coeff = coff.sum(axis=0)

total_weight={}
total_weight_list=[]
for i in range(len(basic_features)):
    total_weight[basic_features[i]]=coeff[i]
    total_weight_list.append((basic_features[i],coeff[i]))
#total_weight_list=sorted(total_weight_list,key=lambda x:x[1],reverse=True)

#trying knn using the weight from the above regression
knn_train_df = train_df.copy()
knn_test_df = test_df.copy()

for feature_value in total_weight_list:
    knn_train_df[feature_value[0]]*=np.sqrt(feature_value[1])
    knn_test_df[feature_value[0]]*=np.sqrt(feature_value[1])

In [35]:
#first edition:
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
#features=list(processing_features)+with_feat
cv_scores=[]
models =[]

for dev_index, val_index in KF:
    dev_set, val_set = knn_train_df.iloc[dev_index,:] , knn_train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[basic_features].as_matrix(), val_set[basic_features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    kn = KN()
    kn.fit(dev_X,dev_y)
    #preds = kn.predict_proba(val_X)
    accr=accuracy_score(kn.predict(val_X),val_y)   
    #cv_scores.append(log_loss(val_y, preds))
    cv_scores.append(accr)  

    #print accr
    break
print np.mean(cv_scores)

0.696585958869


In [36]:
store = "/home/raku/kaggleData/2sigma/knn4/"
knn_train_df.to_json('knn-train.json')
knn_test_df.to_json('knn-test.json')


pickl_file = store+'knn-weigthed-features.pickle'
fileObject = open(pickl_file,'wb') 
pickle.dump(basic_features,fileObject)   
fileObject.close()
print len(basic_features)

16


In [38]:
len(weightFeatures)

139